<a href="https://colab.research.google.com/github/murthy95/colab/blob/master/resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deep Residual learning for image recognition 
This notebook implements the resnet50 architecture discussed in [this](https://https://arxiv.org/pdf/1512.03385.pdf) paper.

The same is shown in the figure attached below 

<img src="https://cdn-images-1.medium.com/max/1600/1*aq0q7gCvuNUqnMHh4cpnIw.png" width=500>

In [0]:
!nvidia-smi

Tue Jun  4 06:59:58 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
#loading the dataset 
import torch 
import torchvision 
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.Resize((224,224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

0it [00:00, ?it/s]

170500096it [00:06, 24378106.21it/s]                               


Files already downloaded and verified


In [0]:
n_classes = 10

#make RESNET50 network
import torch.nn as nn
import torch.nn.functional as F

class simple_residual_block(nn.Module):
  def __init__(self, in_filters, out_filters, is_first, downsample_stride=2):
    super(simple_residual_block, self).__init__()
    
    self.is_first = is_first
    
    if self.is_first:
      self.matching_conv = nn.Conv2d(in_filters, out_filters, 1, 
                                     stride=downsample_stride, padding=0)
      self.conv1 = nn.Conv2d(in_filters, int(out_filters/4), 1, 
                                     stride=1, padding=0)
      self.conv2 = nn.Conv2d(int(out_filters/4), int(out_filters/4), 3, 
                                       stride=downsample_stride, padding=1)
    else:
      self.conv1 = nn.Conv2d(out_filters, int(out_filters/4), 1, 
                                     stride=1, padding=0) 
      self.conv2 = nn.Conv2d(int(out_filters/4), int(out_filters/4), 3, 
                                       stride=1, padding=1)
    self.conv3 = nn.Conv2d(int(out_filters/4), out_filters, 1, 
                                     stride=1, padding=0)
    
  def forward(self, x):
    if self.is_first:
      identity = F.relu(self.matching_conv(x))
    else:
      identity = x
    
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = self.conv3(x)
    x += identity
    return F.relu(x)
  
class Resnet50(nn.Module):
  def __init__(self):
    super(Resnet50, self).__init__()
    
    self.model = nn.Sequential(nn.Conv2d(3, 64, 7, stride=2, padding=3),
                               nn.ReLU(inplace=True),
                               nn.MaxPool2d(3, stride=2, padding=1), 
                               self.make_layers(3, 64, 256, downsample_stride=1),
                               self.make_layers(4, 256, 512),
                               self.make_layers(6, 512, 1024),
                               self.make_layers(3, 1024, 2048))
    self.avgpool = nn.AdaptiveMaxPool2d((1, 1))
    self.linear1 = nn.Linear(2048, 1000)
    self.linear2 = nn.Linear(1000, n_classes)
    
    
    
  def forward(self, x):
    x = self.model(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = F.relu(self.linear1(x))
    x = self.linear2(x)
    return x
  
  def make_layers(self, layers_repeat, in_filters, out_filters, downsample_stride=2):
    blocks = []
    for i in range(layers_repeat):
      if i == 0:
        blocks.append(simple_residual_block(in_filters, out_filters, 
                                            True, downsample_stride))
      else:
        blocks.append(simple_residual_block(in_filters, out_filters, 
                                            False, downsample_stride))
    return nn.Sequential(*blocks)
    

In [5]:
import torch.optim as optim
from torchsummary import summary

#training parameters
n_epochs = 90
alpha = 0.2
batch_size = 64

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

net = Resnet50()
net = net.to(device)
print ('Printing model string : ')
print (summary(net, (3, 224, 224)))

softmax = nn.Softmax(dim=-1)
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

#training continues 
global_counter = 0
for i in range(n_epochs):
  stats_loss = []
  #make a iter object to feed data to the network
  dataiter = iter(trainloader)
  
  for _ in range(len(trainloader)):
    x, y = next(dataiter)
    x = x.to(device)
    y = y.to(device)
    pred_y = net(x)
    
    optimizer.zero_grad()
    _loss = loss(pred_y, y)
    _loss.backward()
    optimizer.step()
    stats_loss.append(_loss.item())
    
    if global_counter % 200 == 0:
      print('Training loss at iteration {} : {}'.format(global_counter, 
                                                        stats_loss[-1]))
      val_loss = 0
      test_data_iter = iter(testloader)
      for j in range(len(testloader)):
        x_test, y_test = next(test_data_iter)
        x_test = x_test.to(device)
        y_test = y_test.to(device)
        val_loss += loss(net(x_test), y_test).item()
     
      print('Test loss at iteration {} : {}'.format(global_counter, 
                                                        val_loss/j))
    global_counter += 1
    
      

Printing model string : 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
              ReLU-2         [-1, 64, 112, 112]               0
         MaxPool2d-3           [-1, 64, 56, 56]               0
            Conv2d-4          [-1, 256, 56, 56]          16,640
            Conv2d-5           [-1, 64, 56, 56]           4,160
            Conv2d-6           [-1, 64, 56, 56]          36,928
            Conv2d-7          [-1, 256, 56, 56]          16,640
simple_residual_block-8          [-1, 256, 56, 56]               0
            Conv2d-9           [-1, 64, 56, 56]          16,448
           Conv2d-10           [-1, 64, 56, 56]          36,928
           Conv2d-11          [-1, 256, 56, 56]          16,640
simple_residual_block-12          [-1, 256, 56, 56]               0
           Conv2d-13           [-1, 64, 56, 56]          16,448
       

KeyboardInterrupt: ignored